In [210]:
import re
from tqdm import tqdm # progress bar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_dataset
import pycld2 as cld2  # language detector
from unidecode import unidecode
import tldextract # domain extractor

In [211]:
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', None)

## load the dataset

In [59]:
wiki = load_dataset("wikimedia/wikipedia", "20231101.az")["train"].to_pandas()
cultrax_df = load_dataset("uonlp/CulturaX", "az")['train'].to_pandas()
print(wiki.shape)
print(cultrax_df.shape)
cultrax_df['source'].value_counts()

source
mC4           4306983
OSCAR-2019     290336
OSCAR-2109     212313
OSCAR-2301     177332
OSCAR-2201      97541
Name: count, dtype: int64

## concat cultrax and wikipedia

In [66]:
# drop wikipedia data to add from the wikipedia itself
cultrax_df = cultrax_df[~cultrax_df['url'].str.contains("wikipedia")]
cultrax_df.drop(columns=['timestamp'], axis=1, inplace=True)
wiki.drop(columns=['id', 'title'], axis=1, inplace=True)
wiki['source'] = 'wiki'
cultrax = pd.concat(objs=[cultrax_df, wiki], axis=0)
print(cultrax.shape)

(5208062, 3)

In [65]:
cultrax['source'].value_counts()

source
mC4           4254852
OSCAR-2019     290336
OSCAR-2109     199080
wiki           196158
OSCAR-2301     172705
OSCAR-2201      94931
Name: count, dtype: int64

## extract the domains from the urls

In [72]:
tqdm.pandas(desc="Processing URLs")
cultrax['domain'] = cultrax['url'].progress_apply(lambda x: tldextract.extract(x).domain)

Processing URLs:   1%|          | 61950/5208062 [00:00<00:16, 314322.52it/s]

Processing URLs: 100%|██████████| 5208062/5208062 [00:15<00:00, 347004.22it/s]


In [82]:
cultrax['domain'].value_counts().index

Index(['', 'wikipedia', 'azadliq', 'trend', 'report', 'azertag', 'publika',
       'amerikaninsesi', 'yenicag', 'sputnik',
       ...
       'fastube', 'al-majalis', 'careersuccess', 'spyaisle',
       'allsportslivestream', 'documentfoundation', 'skygolf', 'hanuur',
       'brasilalemanha', 'linkthe'],
      dtype='object', name='domain', length=29123)

## language detection

In [150]:
def detect_language(text):
    try:
        det = cld2.detect(text)
        is_reliable = det[0]
        lang = det[2][0][1]
        perc = det[2][0][2]
        return is_reliable, lang, perc
    except Exception as e:
        return "pycld2_error", "pycld2_error", "pycld2_error"

In [112]:
tqdm.pandas(desc="Detecting languages")
cultrax[['is_reliable', 'lang', 'perc']] = cultrax['text'].progress_apply(detect_language).apply(pd.Series)

Detecting languages: 100%|██████████| 5208062/5208062 [13:06<00:00, 6618.86it/s] 


In [204]:
def remove_invalid_utf8(text):
    pattern = re.compile(r'(?![ƏəİIıÖöÜüĞğÇçŞş])([^\x00-\x7F]+)')
    cleaned_text = pattern.sub('', text)
    return cleaned_text

In [ ]:
utf_errors = cultrax[cultrax['lang'] == 'pycld2_error']
cultrax = cultrax[cultrax['lang'] != 'pycld2_error']
utf_errors['text'] = utf_errors['text'].apply(remove_invalid_utf8)                          
utf_errors.drop(columns=['is_reliable', 'lang', 'perc'], axis=1, inplace=True)
utf_errors[['is_reliable', 'lang', 'perc']] = utf_errors['text'].apply(detect_language).apply(pd.Series)
cultrax = pd.concat(objs=[cultrax, utf_errors], axis=0)

In [212]:
cultrax['lang'].value_counts()

lang
az              5204194
un                 1249
en                 1184
ru                  470
pycld2_error        253
tr                  226
de                   81
bs                   63
fr                   48
la                   40
da                   26
es                   21
tt                   16
uk                   16
it                   16
hu                   12
pt                   11
ar                   10
sv                    7
pl                    7
bg                    6
el                    6
uz                    6
ka                    4
vo                    4
ro                    4
ja                    4
tk                    3
th                    3
war                   3
aa                    3
hy                    3
sr                    3
id                    3
et                    3
cs                    3
fa                    3
lg                    2
hi                    2
ha                    2
ca                    2
is         

In [214]:
cultrax['is_reliable'].value_counts()

is_reliable
True            5197024
False             10785
pycld2_error        253
Name: count, dtype: int64

In [230]:
a = cultrax[cultrax['lang'] == "pycld2_error"].sample(1)['text'].values[0]
a

'Fəqət bu türk dövlətləri tam bir "islam dövləti" də deyildir. Aradakı fərq\x1flər təməldə və özdə olduğu üçün mühümdür. Türk-islam dövlətinin islam. Gənc qvardiya\nXx əsr Azərbaycan poeziyasının tanınmış nümayəndəsi, xalq şairi, filologiya elmləri doktoru, professor Xəlil Rza Ulutürkün (Xəlil Rza oğlu Xəlilovun) anadan olmasının 80 ILI tamam olur. Gənc qvardiya\nSentyabrın iyirmi doqquzunda məşhur rus yazıçısı Nikolay Ostrovskinin anadan olmasının əlli ILI. Gənc qvardiya'

In [237]:
b = """'Fəqət bu türk dövlətləri tam bir "islam dövləti" də deyildir. Aradakı fərq\x1flər təməldə və özdə olduğu üçün mühümdür. Türk-islam dövlətinin islam. Gənc qvardiya\nXx əsr Azərbaycan poeziyasının tanınmış nümayəndəsi, xalq şairi, filologiya elmləri doktoru, professor Xəlil Rza Ulutürkün (Xəlil Rza oğlu Xəlilovun) anadan olmasının 80 ILI tamam olur. Gənc qvardiya\nSentyabrın iyirmi doqquzunda məşhur rus yazıçısı Nikolay Ostrovskinin anadan olmasının əlli ILI. Gənc qvardiya'"""
cld2.detect(b)

error: input contains invalid UTF-8 around byte 86 (of 533)

In [232]:
remove_invalid_utf8(a)

'Fəqət bu türk dövlətləri tam bir "islam dövləti" də deyildir. Aradakı fərq\x1flər təməldə və özdə olduğu üçün mühümdür. Türk-islam dövlətinin islam. Gənc qvardiya\nXx əsr Azərbaycan poeziyasının tanınmış nümayəndəsi, xalq şairi, filologiya elmləri doktoru, professor Xəlil Rza Ulutürkün (Xəlil Rza oğlu Xəlilovun) anadan olmasının 80 ILI tamam olur. Gənc qvardiya\nSentyabrın iyirmi doqquzunda məşhur rus yazıçısı Nikolay Ostrovskinin anadan olmasının əlli ILI. Gənc qvardiya'

In [238]:
"\x1f" in b

True

In [241]:
c = cultrax[(cultrax['lang'] == "pycld2_error") & ~(cultrax['text'].str.contains("\x1f"))]

In [244]:
c.sample(1)['text'].values[0]

'Şəki rayonunda balıqçılığın inkişafına diqqət ildən-ilə artır. Hazırda rayonun Aydınbulaq, Aşağı Küngüt, Qırxbulaq və digər kəndlərinin ərazilərində 7 özəl balıqçılıq təsərrüfatı fəaliyyət göstərir. Şəhər icra hakimiyyətinin aqrar məsəl\x06 ...'

In [246]:
def remove_pattern(text):
    pattern = r'\\x[0-9a-fA-F]{2}'
    return re.sub(pattern, '', text)

text = 'Şəki rayonunda balıqçılığın inkişafına diqqət ildən-ilə artır. Hazırda rayonun Aydınbulaq, Aşağı Küngüt, Qırxbulaq və digər kəndlərinin ərazilərində 7 özəl balıqçılıq təsərrüfatı fəaliyyət göstərir. Şəhər icra hakimiyyətinin aqrar məsəl\x06 ...'
cleaned_text = remove_pattern(text)
print(cleaned_text)

Şəki rayonunda balıqçılığın inkişafına diqqət ildən-ilə artır. Hazırda rayonun Aydınbulaq, Aşağı Küngüt, Qırxbulaq və digər kəndlərinin ərazilərində 7 özəl balıqçılıq təsərrüfatı fəaliyyət göstərir. Şəhər icra hakimiyyətinin aqrar məsəl ...
